In [1]:
!pip install langchain_openai langchain_community langchain pymysql chromadb -q langchain_community

In [25]:
db_user = "xxx"
db_password = "xxx"
db_host = "xxx"
db_name = "xxx"
from langchain_community.utilities.sql_database import SQLDatabase
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")
print(db.dialect)
print(db.get_usable_table_names())
print(db.table_info)

mysql
['中信金_損益表', '中信金_現金流量表', '中信金_資產負債表', '元大金_損益表', '元大金_現金流量表', '元大金_資產負債表', '兆豐金_損益表', '兆豐金_現金流量表', '兆豐金_資產負債表', '台新金_損益表', '台新金_現金流量表', '台新金_資產負債表', '合庫金_損益表', '合庫金_現金流量表', '合庫金_資產負債表', '國泰金_損益表', '國泰金_現金流量表', '國泰金_資產負債表', '國票金_損益表', '國票金_現金流量表', '國票金_資產負債表', '富邦金_損益表', '富邦金_現金流量表', '富邦金_資產負債表', '新光金_損益表', '新光金_現金流量表', '新光金_資產負債表', '永豐金_損益表', '永豐金_現金流量表', '永豐金_資產負債表', '玉山金_損益表', '玉山金_現金流量表', '玉山金_資產負債表', '第一金_損益表', '第一金_現金流量表', '第一金_資產負債表', '華南金_損益表', '華南金_現金流量表', '華南金_資產負債表', '開發金_損益表', '開發金_現金流量表', '開發金_資產負債表']

CREATE TABLE `中信金_損益表` (
	`代號` BIGINT, 
	`名稱` VARCHAR(50) COLLATE utf8mb4_unicode_ci, 
	`年_月` VARCHAR(20) COLLATE utf8mb4_unicode_ci, 
	`營業收入淨額` BIGINT, 
	`營業費用` BIGINT, 
	`利息收入` BIGINT, 
	`採權益法之關聯企業及合資損益之份額` BIGINT, 
	`除列按攤銷後成本衡量金融資產淨損益` BIGINT, 
	`金融資產重分類淨損益` BIGINT, 
	`稅前淨利` BIGINT, 
	`所得稅費用` BIGINT, 
	`繼續營業單位損益` BIGINT, 
	`停業單位損益` BIGINT, 
	`合併前非屬共同控制股權損益` BIGINT, 
	`其他損益調整項_非常項目及累計影響數` BIGINT, 
	`合併總損益` BIGINT, 
	`不重分類至損益之項目_OCI` BIGINT, 
	`後續可能重分類至損益之項目_OCI` BIGIN

In [26]:
import os
os.environ['OPENAI_API_KEY']= "your api key"

In [27]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
llm = ChatOpenAI(model="gpt-4")

#生成SQL語法
generate_query = create_sql_query_chain(llm, db)

In [28]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
#執行SQL
execute_query = QuerySQLDataBaseTool(db=db)

In [29]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
     """Given the following user question, corresponding SQL query, and SQL result, answer the user question.
        用繁體中文且通順的句子回答。
 Question: {question}
 SQL Query: {query}
 SQL Result: {result}
 Answer: """
 )

rephrase_answer = answer_prompt | llm | StrOutputParser()

#把generate_query, execute_query, rephrase_answer串起來
# chain = (
#     RunnablePassthrough.assign(query=generate_query).assign(
#         result=itemgetter("query") | execute_query
#     ) | rephrase_answer
# )

# chain.invoke({"question": "給我中信金2023的每股盈餘"})


In [30]:
examples = [
    {
        "input": "請給我中信金2022年12月的營業費用",
        "query": """SELECT 營業費用 
                    FROM 中信金_損益表 
                    WHERE 代號 = '2891' AND 年_月 = 'Dec-22'"""
    },
    {
        "input": "我需要元大金2021年的現金流量總額.",
        "query": """SELECT SUM(本期產生現金流量) AS 本期產生現金流量
                    FROM 元大金_現金流量表
                    WHERE 代號 = '2885' AND 年_月 LIKE '21%'
    """
    },
        {
        "input": "請給我國票金2023和2022的資產總額和負債總額比較",
        "query": """SELECT 
                        SUM(CASE WHEN 年_月 LIKE '%Dec-23' THEN 資產總額 ELSE 0 END) AS 2023年資產總額,
                        SUM(CASE WHEN 年_月 LIKE '%Dec-22' THEN 資產總額 ELSE 0 END) AS 2022年資產總額,
                        SUM(CASE WHEN 年_月 LIKE '%Dec-23' THEN 負債總額 ELSE 0 END) AS 2023年負債總額,
                        SUM(CASE WHEN 年_月 LIKE '%Dec-22' THEN 負債總額 ELSE 0 END) AS 2022年負債總額
                    FROM 國票金_資產負債表
                    WHERE 代號 = '2889';
    """
    }
 ]


In [31]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,FewShotChatMessagePromptTemplate,PromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}\nSQLQuery:"),
        ("ai", "{query}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
     # input_variables=["input","top_k"],
    input_variables=["input"],
)
# print(few_shot_prompt.format(input1="How many products are there?"))


In [32]:
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma()
vectorstore.delete_collection()
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    vectorstore,
    k=2,
    input_keys=["input"],
)
example_selector.select_examples({"input": "how many employees we have?"})
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    input_variables=["input","top_k"],
)
print(few_shot_prompt.format(input="我需要中信金2020九月的現金流量"))

Human: 我需要元大金2021年的現金流量總額.
SQLQuery:
AI: SELECT SUM(本期產生現金流量) AS 本期產生現金流量
                    FROM 元大金_現金流量表
                    WHERE 代號 = '2885' AND 年_月 LIKE '21%'
    
Human: 請給我中信金2022年12月的營業費用
SQLQuery:
AI: SELECT 營業費用 
                    FROM 中信金_損益表 
                    WHERE 代號 = '2891' AND 年_月 = 'Dec-22'


In [11]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "您是一位MySQL專家。根據輸入的問題，創建一個語法正確的MySQL查詢來運行。除非另有具體指定。\n\n以下是相關的表信息：{table_info}\n\n下面是一些問題及其相應的SQL查詢的示例。\n\n"),
        few_shot_prompt,
        ("human", "{input}")
    ]
)
print(final_prompt.format(input="中信金2023年的每股盈餘",table_info="some table info"))

System: 您是一位MySQL專家。根據輸入的問題，創建一個語法正確的MySQL查詢來運行。除非另有具體指定。

以下是相關的表信息：some table info

下面是一些問題及其相應的SQL查詢的示例。


Human: 請給我中信金2022年12月的營業費用
SQLQuery:
AI: SELECT 營業費用 
                    FROM 中信金_損益表 
                    WHERE 代號 = '2891' AND 年_月 = 'Dec-22'
Human: 請給我國票金2023和2022的資產總額和負債總額比較
SQLQuery:
AI: SELECT 
                        SUM(CASE WHEN 年_月 LIKE '%Dec-23' THEN 資產總額 ELSE 0 END) AS 2023年資產總額,
                        SUM(CASE WHEN 年_月 LIKE '%Dec-22' THEN 資產總額 ELSE 0 END) AS 2022年資產總額,
                        SUM(CASE WHEN 年_月 LIKE '%Dec-23' THEN 負債總額 ELSE 0 END) AS 2023年負債總額,
                        SUM(CASE WHEN 年_月 LIKE '%Dec-22' THEN 負債總額 ELSE 0 END) AS 2022年負債總額
                    FROM 國票金_資產負債表
                    WHERE 代號 = '2889';
    
Human: 中信金2023年的每股盈餘


In [33]:
from operator import itemgetter
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
import pandas as pd

In [34]:
def get_table_details():
    # 將 CSV 文件讀取到 DataFrame 中
    table_description = pd.read_csv("/Users/rex/LLM/graduation_project/final_project_table_info_simple2.csv")
    table_details = []

    # 遍歷 DataFrame 行以創建包含表格詳細信息的字符串
    for index, row in table_description.iterrows():
        table_name = str(row['Table'])
        table_description = str(row['Description'])
        table_details.append(f"表名: {table_name}\n表描述: {table_description}\n\n")

    return "".join(table_details)

class Table(BaseModel):
    """SQL 數據庫中的表格。"""

    name: str = Field(description="SQL 數據庫中表格的名稱。")

table_details = get_table_details()
print(table_details)


表名: 中信金_損益表
表描述: 中信金的損益表，顯示公司在期間內所獲得的收入和支付的費用，以及該期間的淨利潤或淨損失

表名: 元大金_損益表
表描述: 元大金的損益表，顯示公司在期間內所獲得的收入和支付的費用，以及該期間的淨利潤或淨損失

表名: 兆豐金_損益表
表描述: 兆豐金的損益表，顯示公司在期間內所獲得的收入和支付的費用，以及該期間的淨利潤或淨損失

表名: 台新金_損益表
表描述: 台新金的損益表，顯示公司在期間內所獲得的收入和支付的費用，以及該期間的淨利潤或淨損失

表名: 合庫金_損益表
表描述: 合庫金的損益表，顯示公司在期間內所獲得的收入和支付的費用，以及該期間的淨利潤或淨損失

表名: 國泰金_損益表
表描述: 國泰金的損益表，顯示公司在期間內所獲得的收入和支付的費用，以及該期間的淨利潤或淨損失

表名: 國票金_損益表
表描述: 國票金的損益表，顯示公司在期間內所獲得的收入和支付的費用，以及該期間的淨利潤或淨損失

表名: 富邦金_損益表
表描述: 富邦金的損益表，顯示公司在期間內所獲得的收入和支付的費用，以及該期間的淨利潤或淨損失

表名: 新光金_損益表
表描述: 新光金的損益表，顯示公司在期間內所獲得的收入和支付的費用，以及該期間的淨利潤或淨損失

表名: 永豐金_損益表
表描述: 永豐金的損益表，顯示公司在期間內所獲得的收入和支付的費用，以及該期間的淨利潤或淨損失

表名: 玉山金_損益表
表描述: 玉山金的損益表，顯示公司在期間內所獲得的收入和支付的費用，以及該期間的淨利潤或淨損失

表名: 第一金_損益表
表描述: 第一金的損益表，顯示公司在期間內所獲得的收入和支付的費用，以及該期間的淨利潤或淨損失

表名: 華南金_損益表
表描述: 華南金的損益表，顯示公司在期間內所獲得的收入和支付的費用，以及該期間的淨利潤或淨損失

表名: 開發金_損益表
表描述: 開發金的損益表，損益表是用來顯示公司在期間內所獲得的收入和支付的費用，以及該期間的淨利潤或淨損失

表名: 中信金_現金流量表
表描述: 中信金的現金流量表，顯示公司在期間內的現金流入和流出情況，並分類為營運、投資和融資活動

表名: 元大金_現金流量表
表描述: 元大金的現金流量表，顯示公司在期間內的現金流入和流出情況，並分類為營運、投資和融資活動

表名: 兆豐金_現金流量

In [48]:
table_details_prompt = f"""返回所有可能與用戶問題相關的SQL表的名稱。
Return the names of ALL the SQL tables that MIGHT be relevant to the user question.

這些表格是台灣十四家金控股2019-2023年的損益表、現金流量表、資產負債表：
損益表重要欄位有：營業收入淨額、營業費用、利息收入、稅前淨利、每股盈餘、加權平均股數、發放特別股股息、稀釋稅後淨利。
現金流量表重要欄位有：營業收入淨額、營業費用、利息收入、稅前淨利、每股盈餘、加權平均股數、發放特別股股息、稀釋稅後淨利。
資產負債表重要欄位有：現金及約當現金、應收帳款及票據、不動產廠房及設備、流動資產、非流動資產、資產總額、流動負債、非流動負債、負債總額、普通股股本、特別股股本、預收股款、股本、資本公積合計、保留盈餘、庫藏股票帳面值、股東權益總額。


{table_details}

記得包括所有可能相關的表格，當需要多個表格時，請回傳多個即使你不確定它們是否需要。
Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""

table_chain = create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt)
tables = table_chain.invoke({"input": "請給我中信金2023三月的每股盈餘"})
tables


[Table(name='中信金_損益表')]

In [42]:
def get_tables(tables: List[Table]) -> List[str]:
    tables  = [table.name for table in tables]
    return tables

select_table = {"input": itemgetter("question")} | create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt) | get_tables
# select_table.invoke({"question": "請給我中信金2023三月的資產總額"})


['中信金_資產負債表']

In [53]:
chain = (
RunnablePassthrough.assign(table_names_to_use=select_table) |
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)
chain.invoke({"question": "請給我中信金2020九月的每股盈餘"})

'中信金在2020年九月的每股盈餘為2.44元。這表示在該月份中，中信金每一股的平均盈利為2.44元。'